01. ByteTracker 
- very easy to implement since its integrated to supervision - the open-source computer vision toolkit by Roboflow


In [ ]:
#imports and paths
import os
from inference import get_model
import supervision as sv
from ultralytics import YOLO
import numpy as np
import sys
sys.path.append(os.getenv("PROJECT_PATH"))

VIDEO_PATH = "../../data/videos/old10s.mp4"
PLAYER_DETECTION_MODEL_ID = 'football-players-detection-3zvbc/2'
ROBOFLOW_API_KEY = os.getenv("ROBOFLOW_API_KEY")
PLAYER_DETECTION_MODEL = get_model(PLAYER_DETECTION_MODEL_ID, ROBOFLOW_API_KEY)
tracker = sv.ByteTrack()


ByteTracker

In [ ]:
#save results in video
tracker = sv.ByteTrack()

bounding_box_annotator = sv.BoundingBoxAnnotator()
label_annotator = sv.LabelAnnotator()

def callback(frame: np.ndarray, index: int) -> np.ndarray:
    results = PLAYER_DETECTION_MODEL.infer(frame,confidence=0.3)[0]
    detections = sv.Detections.from_inference(results)
    
    detections = tracker.update_with_detections(detections)

    labels = [f"#{tracker_id}" for tracker_id in detections.tracker_id]

    annotated_frame = bounding_box_annotator.annotate(
        scene=frame.copy(), detections=detections)
    annotated_frame = label_annotator.annotate(
        scene=annotated_frame, detections=detections, labels=labels)
    return annotated_frame

sv.process_video(
    source_path= VIDEO_PATH,
    target_path= "../../data/tracker_outputs/bytetracker.mp4",
    callback=callback
)

In [ ]:
#save results in MOT format
def save_mot_format(tracker, video_path, output_path):
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w') as f:
        for frame_idx, frame in enumerate(sv.get_video_frames_generator(video_path)):
            results = PLAYER_DETECTION_MODEL.infer(frame, confidence=0.3)[0]
            detections = sv.Detections.from_inference(results)
            detections = tracker.update_with_detections(detections)
            for det, track_id in zip(detections.xyxy, detections.tracker_id):
                x1, y1, x2, y2 = det
                w, h = x2 - x1, y2 - y1
                # MOT format: frame, id, x, y, w, h, 1, -1, -1, -1
                f.write(f"{frame_idx+1},{track_id},{x1:.2f},{y1:.2f},{w:.2f},{h:.2f},1,-1,-1,-1\n")

save_mot_format(tracker, VIDEO_PATH, "../../data/tracker_outputs/bytetracker_mot.txt")

In [ ]:
#MOT Evaluation
#todo